In [65]:
# Classification template

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values


dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [41]:
# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder() # Encode countries
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) # Will encode the 3 country strings to 0,1,2

labelencoder_X_2 = LabelEncoder() # Encode gender
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) # Will encode country gender to 0,1


In [42]:
# Don't need to onehotencode gender because its binary

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# Encoding the Dependent Variable
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

/Users/natepill/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/natepill/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [43]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [44]:
# Feature Scaling
# Very important for NN calculations
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [45]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [46]:
# Initialize the Classifier ANN
classifier = Sequential()

# Adding input layer AND first hidden layer
# output_dim = (11 inputs nodes + 1 output node )/2 = 6
# rectifier activation

# classifier.add(Dense(output_dim = 6, init='uniform', activation='relu', input_dim=11))
# Same as above, just updated for Keras 2 API
classifier.add(Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform"))

'''
Dense parameters:
output_dim: number of nodes you want to add to the next HIDDEN layer
Determining the number of nodes in the hidden layer is an art, but one tip
could be to have the number of nodes in the hidden layer be equal to the average of the nodes in 
the input layer and output layer. You could also determine this number by doing some form of parameter tuning

init: Will randomly initizalize weights to 0, using the glorot_uniform function be default

activation: What activation function to use

input_dim: Number of input nodes in our INPUT layer, so we only need to specify this parameter ONCE
'''


'\nDense parameters:\noutput_dim: number of nodes you want to add to the next HIDDEN layer\nDetermining the number of nodes in the hidden layer is an art, but one tip\ncould be to have the number of nodes in the hidden layer be equal to the average of the nodes in \nthe input layer and output layer. You could also determine this number by doing some form of parameter tuning\n\ninit: Will randomly initizalize weights to 0, using the glorot_uniform function be default\n\nactivation: What activation function to use\n\ninput_dim: Number of input nodes in our INPUT layer, so we only need to specify this parameter ONCE\n'

In [47]:
# Adding second hidden layer
classifier.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))

# Adding output layer
# Sigmoid activation will output possibility of the classification
# We can use softmax activation function that is like sigmoid, but works for multiclass-classification
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

In [50]:
# Compiling the ANN
# Compiling is the "training" or "learning" process for our ANN
# Stochastic Gradient descent will utilze a logartimic loss function NOT MSE, this is based on the 
# sigmoid activation of our output layer
# loss function for binary logarithmic: binary_crossentropy, if not binary, then: categorical_crossentropy
classifier.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
'''
Compile parameters:

optimizer: algorithm to find the optimial set of weights - How we tune our weights - ie: Adam Gradient Descent Algo.

loss: Loss function to determine the error between ypred and ytrue

metrics: list of metrics to be evaluated by the model and then improved upon, typically 'accuracy'

epochs: number of training iterations, one full cycle of foward and back propagations
'''


"\nCompile parameters:\n\noptimizer: algorithm to find the optimial set of weights - How we tune our weights - ie: Adam Gradient Descent Algo.\n\nloss: Loss function to determine the error between ypred and ytrue\n\nmetrics: list of metrics to be evaluated by the model and then improved upon, typically 'accuracy'\n\nepochs: number of training iterations, one full cycle of foward and back propagations\n"

In [51]:
# Fitting the ANN to the training set
classifier.fit(X_train, y_train, batch_size = 10, epochs=100)

Epoch 1/100
7500/7500 [==============================] - 1s 159us/step - loss: 0.4921 - acc: 0.7957
Epoch 2/100
7500/7500 [==============================] - 1s 93us/step - loss: 0.4298 - acc: 0.7963
Epoch 3/100
7500/7500 [==============================] - 1s 92us/step - loss: 0.4255 - acc: 0.7963
Epoch 4/100
7500/7500 [==============================] - 1s 116us/step - loss: 0.4209 - acc: 0.8040 0s - loss: 0.4264 - acc
Epoch 5/100
7500/7500 [==============================] - 1s 94us/step - loss: 0.4180 - acc: 0.8224
Epoch 6/100
7500/7500 [==============================] - 1s 95us/step - loss: 0.4159 - acc: 0.8257
Epoch 7/100
7500/7500 [==============================] - 1s 105us/step - loss: 0.4136 - acc: 0.8300
Epoch 8/100
7500/7500 [==============================] - 1s 109us/step - loss: 0.4131 - acc: 0.8316
Epoch 9/100
7500/7500 [==============================] - 1s 102us/step - loss: 0.4116 - acc: 0.8315
Epoch 10/100
7500/7500 [==============================] - 1s 108us/step - loss: 

7500/7500 [==============================] - 1s 152us/step - loss: 0.4001 - acc: 0.8347
Epoch 79/100
7500/7500 [==============================] - 1s 132us/step - loss: 0.4000 - acc: 0.8368
Epoch 80/100
7500/7500 [==============================] - 1s 142us/step - loss: 0.4007 - acc: 0.8351 0s - loss: 0.4003 - acc: 0.8 - ETA: 0s - loss: 0.3959 - acc: 0.836 - ETA: 0s - loss: 0.4000 - acc: 0.835
Epoch 81/100
7500/7500 [==============================] - 1s 142us/step - loss: 0.4004 - acc: 0.8351
Epoch 82/100
7500/7500 [==============================] - 1s 146us/step - loss: 0.4001 - acc: 0.8359
Epoch 83/100
7500/7500 [==============================] - 1s 137us/step - loss: 0.3998 - acc: 0.8351 0s - loss: 0.3898 -
Epoch 84/100
7500/7500 [==============================] - 1s 143us/step - loss: 0.3998 - acc: 0.8357
Epoch 85/100
7500/7500 [==============================] - 1s 159us/step - loss: 0.4002 - acc: 0.8352
Epoch 86/100
7500/7500 [==============================] - 1s 149us/step - loss: 

In [52]:
# Fitting classifier to the Training set
# Create your classifier here

# Predicting the Test set results
y_pred = classifier.predict(X_test) # Returns ONLY probabilities of people leaving the bank

# Convert probabilities to True/False using a threshold value in order to evaluate the model. Threshold value will
# be dependent on the sensitivity of the results

y_pred = (y_pred > 0.5) # Compares each value to 0.5 threshold and replaces with bool


# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [62]:
cm

array([[1923,   68],
       [ 326,  183]])

In [57]:
1923 + 183

2106

In [58]:
1923 + 183 + 68 + 326

2500

In [60]:
2106/2500 # Accuracy score

0.8424

### Homework problem

In [ ]:
current_client = []